In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sephora-products-and-skincare-reviews/product_info.csv
/kaggle/input/sephora-products-and-skincare-reviews/reviews_250_500.csv
/kaggle/input/sephora-products-and-skincare-reviews/reviews_1500_end.csv
/kaggle/input/sephora-products-and-skincare-reviews/reviews_1000_1500.csv
/kaggle/input/sephora-products-and-skincare-reviews/reviews_500_750.csv
/kaggle/input/sephora-products-and-skincare-reviews/reviews_750_1000.csv
/kaggle/input/sephora-products-and-skincare-reviews/reviews_0_250.csv


#EDA

In [2]:
df_products = pd.read_csv('/kaggle/input/sephora-products-and-skincare-reviews/product_info.csv')

In [3]:
df_products.shape
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8494 entries, 0 to 8493
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   product_id          8494 non-null   object 
 1   product_name        8494 non-null   object 
 2   brand_id            8494 non-null   int64  
 3   brand_name          8494 non-null   object 
 4   loves_count         8494 non-null   int64  
 5   rating              8216 non-null   float64
 6   reviews             8216 non-null   float64
 7   size                6863 non-null   object 
 8   variation_type      7050 non-null   object 
 9   variation_value     6896 non-null   object 
 10  variation_desc      1250 non-null   object 
 11  ingredients         7549 non-null   object 
 12  price_usd           8494 non-null   float64
 13  value_price_usd     451 non-null    float64
 14  sale_price_usd      270 non-null    float64
 15  limited_edition     8494 non-null   int64  
 16  new   

In [4]:
df_products.isna().sum(axis=0)

product_id               0
product_name             0
brand_id                 0
brand_name               0
loves_count              0
rating                 278
reviews                278
size                  1631
variation_type        1444
variation_value       1598
variation_desc        7244
ingredients            945
price_usd                0
value_price_usd       8043
sale_price_usd        8224
limited_edition          0
new                      0
online_only              0
out_of_stock             0
sephora_exclusive        0
highlights            2207
primary_category         0
secondary_category       8
tertiary_category      990
child_count              0
child_max_price       5740
child_min_price       5740
dtype: int64

In [5]:
df_products.groupby(['brand_name'])['product_name'].count().sort_values(ascending=False).head(10)

brand_name
SEPHORA COLLECTION         352
CLINIQUE                   179
Dior                       136
tarte                      131
NEST New York              115
Bumble and bumble          110
Kérastase                  108
TOM FORD                   100
Charlotte Tilbury           99
Anastasia Beverly Hills     95
Name: product_name, dtype: int64

In [6]:
#popular product_category in each brand

tmp = pd.DataFrame(df_products.groupby(['brand_name','primary_category'])['product_id'].count().reset_index())
tmp['product_rank'] = tmp.groupby(['brand_name'])['product_id'].rank(method = 'min',ascending = False)
tmp[tmp.product_rank == 1]

,brand_name,primary_category,product_id,product_rank
0,19-69,Fragrance,14,1.0
1,54 Thrones,Bath & Body,4,1.0
2,ABBOTT,Fragrance,13,1.0
3,AERIN,Fragrance,22,1.0
6,ALTERNA Haircare,Hair,45,1.0
...,...,...,...,...
618,philosophy,Bath & Body,1,1.0
619,rms beauty,Makeup,11,1.0
620,shu uemura,Hair,43,1.0
621,stila,Makeup,10,1.0


In [7]:
#popular product in each brand
tmp1 = df_products[['brand_name','product_name','loves_count']]
tmp1['rank']= tmp1.groupby(['brand_name'])['loves_count'].rank(method='min',ascending=False)
tmp1

/tmp/ipykernel_20/3412678004.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp1['rank']= tmp1.groupby(['brand_name'])['loves_count'].rank(method='min',ascending=False)


,brand_name,product_name,loves_count,rank
0,19-69,Fragrance Discovery Set,6320,1.0
1,19-69,La Habana Eau de Parfum,3827,2.0
2,19-69,Rainbow Bar Eau de Parfum,3253,3.0
3,19-69,Kasbah Eau de Parfum,3018,4.0
4,19-69,Purple Haze Eau de Parfum,2691,5.0
...,...,...,...,...
8489,Yves Saint Laurent,Couture Clutch Eyeshadow Palette,2790,61.0
8490,Yves Saint Laurent,L'Homme Eau de Parfum,2319,62.0
8491,Yves Saint Laurent,Mon Paris Eau de Parfum Gift Set,1475,63.0
8492,Yves Saint Laurent,Y Eau de Parfum Gift Set,840,64.0


In [8]:
#Average price of each product category
tmp = df_products[['primary_category','secondary_category','tertiary_category','price_usd']]
tmp1 = pd.DataFrame(tmp.groupby(['primary_category','secondary_category','tertiary_category'])['price_usd'].min().reset_index())
tmp1['max_price']=pd.DataFrame(tmp.groupby(['primary_category','secondary_category','tertiary_category'])['price_usd'].max().reset_index())['price_usd']
tmp1['mean_price']=pd.DataFrame(tmp.groupby(['primary_category','secondary_category','tertiary_category'])['price_usd'].mean().reset_index())['price_usd']
tmp1.rename(columns={'price_usd':'min_price'})

,primary_category,secondary_category,tertiary_category,min_price,max_price,mean_price
0,Bath & Body,Bath & Shower,Bath Soaks & Bubble Bath,3.0,82.0,29.142857
1,Bath & Body,Bath & Shower,Body Wash & Shower Gel,6.0,70.0,30.705882
2,Bath & Body,Bath & Shower,Scrub & Exfoliants,18.0,155.0,43.692308
3,Bath & Body,Body Care,Cellulite & Stretch Marks,40.0,77.0,62.625000
4,Bath & Body,Body Care,Deodorant & Antiperspirant,8.0,96.0,28.920000
...,...,...,...,...,...,...
130,Tools & Brushes,Brushes & Applicators,Facial Cleansing Brushes,14.0,16.0,15.000000
131,Tools & Brushes,Brushes & Applicators,Sponges & Applicators,10.0,14.0,12.000000
132,Tools & Brushes,Hair Tools,Accessories,15.0,45.0,30.000000
133,Tools & Brushes,Hair Tools,Brushes & Combs,7.0,74.0,26.000000


In [9]:
df_products.ingredients.str.replace('[', '', regex=False).str.replace(']', '', regex=False).str.split(',',expand=True)

,0,1,2,3,4,5,6,7,8,9,...,710,711,712,713,714,715,716,717,718,719
0,'Capri Eau de Parfum:','Alcohol Denat. (SD Alcohol 39C),Parfum (Fragrance) D-Limonene,Linalool,Benzyl Salicylate,Ethylhexyl Methoxycinnamate,Butyl Methoxydibenzoylmethane,Ethylhexyl Salicylate,Benzl Benzoate,Citral,...,None,None,None,None,None,None,None,None,None,None
1,'Alcohol Denat. (SD Alcohol 39C),Parfum (Fragrance) Ethylhexyl Methoxycinnamate,Ethylhexyl Salicylate,Butyl Methoxydibenzoylmethane,Benzyl Alcohol,Benzyl Benzoate,Benzyl Cinnamate,Cinnamal,Citral,Coumarin Limonene,...,None,None,None,None,None,None,None,None,None,None
2,'Alcohol Denat. (SD Alcohol 39C),Parfum (Fragrance) D-Limonene,Ethylhexyl Methoxycinnamate,Butyl Methoxydibenzoylmethane,Ethylhexyl Salicylate,Hexyl Cinnamal Citronellol,Linalool,Coumarin,Eugenol,Citral,...,None,None,None,None,None,None,None,None,None,None
3,'Alcohol Denat. (SD Alcohol 39C),Parfum (Fragrance) Coumarin,Ethylhexyl Methoxycinnamate,Butyl Methoxydibenzoylmethane,Ethylhexyl Salicylate,D-Limonene,Eugenol,Linalool,Citronellol,Geraniol,...,None,None,None,None,None,None,None,None,None,None
4,'Alcohol Denat. (SD Alcohol 39C),Parfum (Fragrance) Octoxynol-10,D-Limonene,Ethylhexyl Methoxycinnamate,Butyl Methoxydibenzoylmethane,Ethylhexyl Salicylate,Geraniol,Linalool,Coumarin,Farnesol,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8489,'Talc,Synthetic Fluorphlogopite,Triethylhexanoin,Octyldodecyl Stearoyl Stearate,Ci 77891 / Titanium Dioxide,Caprylyl Glycol,Ci 77491,Ci 77499 / Iron Oxides,Dehydroacetic Acid,Phenoxyethanol,...,None,None,None,None,None,None,None,None,None,None
8490,'Alcohol,Aqua / Water / Eau,Parfum / Fragrance,Limonene,Linalool,Coumarin,Butyl Methoxydibenzoylmethane,Alpha-Isomethyl Ionone,Geraniol,Citral,...,None,None,None,None,None,None,None,None,None,None
8491,'Mon Paris Eau de Parfum:','Alcohol,Parfum/Fragrance,Aqua/Water,Benzyl Salicylate,Linalool,Benzyl Alcohol,Hydroxycitronellal,Ethylhexyl Methoxycinnamate,Limonene,...,None,None,None,None,None,None,None,None,None,None
8492,'Alcohol,Parfum/Fragrance,Aqua/Water,Limonene,Linalool,Butyl Methoxydibenzoylmethane,Citral,Citronellol,Alpha-Isomethyl Ionone,Geraniol,...,None,None,None,None,None,None,None,None,None,None


#Sentiment Analysis

In [10]:
working_dir = '../input/sephora-products-and-skincare-reviews/'
filename ='reviews_0_250.csv'
review_df_0_250 = pd.read_csv(working_dir + filename) 


/tmp/ipykernel_20/3114123174.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  review_df_0_250 = pd.read_csv(working_dir + filename)


In [11]:
reviews_250 = pd.read_csv('/kaggle/input/sephora-products-and-skincare-reviews/reviews_0_250.csv',low_memory = False)
reviews_500 = pd.read_csv('/kaggle/input/sephora-products-and-skincare-reviews/reviews_250_500.csv',low_memory = False)
reviwes_750 = pd.read_csv('/kaggle/input/sephora-products-and-skincare-reviews/reviews_500_750.csv',low_memory = False)
reviwes_1000 = pd.read_csv('/kaggle/input/sephora-products-and-skincare-reviews/reviews_750_1000.csv',low_memory = False)
reviwes_1500 = pd.read_csv('/kaggle/input/sephora-products-and-skincare-reviews/reviews_1000_1500.csv',low_memory = False)


In [12]:
reviews = pd.concat([reviews_250, reviews_500, reviwes_750, reviwes_1000 , reviwes_1500])
reviews = reviews[['review_text' , 'is_recommended' , 'rating']]
reviews

,review_text,is_recommended,rating
0,I use this with the Nudestix “Citrus Clean Bal...,1.0,5
1,I bought this lip mask after reading the revie...,0.0,1
2,My review title says it all! I get so excited ...,1.0,5
3,I’ve always loved this formula for a long time...,1.0,5
4,"If you have dry cracked lips, this is a must h...",1.0,5
...,...,...,...
119312,I LOVE this stuff! It works amazingly well at ...,1.0,5
119313,love this micellar water from Milk! it removes...,1.0,5
119314,I loveeeeee Milk!!! Just discovered this brand...,1.0,5
119315,"My new favorite makeup remover. First of all, ...",1.0,5


In [13]:
reviews.rename(columns = {'is_recommended':'label'},inplace=True)
reviews.isna().sum()

review_text      1557
label          190157
rating              0
dtype: int64

In [14]:
reviews.isnull().sum()
reviews.dropna(inplace=True)

In [15]:
reviews.label.astype(int)
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1059445 entries, 0 to 119316
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   review_text  1059445 non-null  object 
 1   label        1059445 non-null  float64
 2   rating       1059445 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 32.3+ MB


In [16]:
num_neg = pd.DataFrame(reviews.label.value_counts().reset_index()).iloc[1,1]
num_neg

168797

In [17]:
reviews_sample = reviews.groupby('label').apply(lambda x : x.sample(num_neg))
reviews_sample.label.value_counts()

0.0    168797
1.0    168797
Name: label, dtype: int64

#text pre-processing

In [18]:
from sklearn.utils import shuffle
import string
string.punctuation

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [19]:
reviews_sample= shuffle(reviews_sample)
reviews_sample.reset_index(drop=True, inplace= True)
reviews_sample

,review_text,label,rating
0,"I was gifted by Sephora, a complementary sampl...",0.0,1
1,I really wanted to like this eye cream. It ac...,0.0,2
2,"Very nice serum, very hydrating for the skin, ...",1.0,5
3,I received this product complimentary from Inf...,1.0,4
4,"This product was nice, it takes some time to s...",0.0,3
...,...,...,...
337589,I’ve only used this for a short amount of time...,1.0,5
337590,"This works amazing, I got as a sample and I pu...",1.0,5
337591,I got this as a free sample and I love how it ...,1.0,5
337592,I don’t typically use a toner but I added this...,0.0,3


In [20]:
reviews_sample.to_csv('/kaggle/working/reviews_sample.csv')

In [21]:
reviews_sample = pd.read_csv('/kaggle/working/reviews_sample.csv')

In [22]:
#number of words in each review
reviews_sample['text_word_count']=reviews_sample['review_text'].apply(lambda x:len(x.split()))
reviews_sample['text_word_count']

0         110
1          67
2          12
3         332
4          40
         ... 
337589     51
337590     42
337591     30
337592     33
337593     53
Name: text_word_count, Length: 337594, dtype: int64

To prepare the text data for the model building we perform text preprocessing. It is the very first step of NLP projects. Some of the preprocessing steps are:

Removing punctuations \
Removing URLs\
Removing Stop words\
Lower casing\
Tokenization\
Stemming\
Lemmatization

In [23]:
# removing punctuations
def remove_punc(text):
    
    clean_text = "".join([i for i in text if i not in string.punctuation])
    return clean_text

reviews_sample['clean_text']  = reviews_sample['review_text'].apply(lambda x: remove_punc(x))


In [24]:
# removing HTML text
from bs4 import BeautifulSoup
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()
reviews_sample['clean_text']  = reviews_sample['clean_text'].apply(lambda x: strip_html(x))


In [25]:
# converting to lower case
reviews_sample['clean_text']  = reviews_sample['clean_text'].apply(lambda x: x.lower())

In [26]:
# Tokenize text
reviews_sample['tokenized_text']  = reviews_sample['clean_text'].apply(lambda x: x.split())

In [27]:
#stop words removal
import nltk 
stopwords = nltk.corpus.stopwords.words('english')
stopwords[0:10]
def remove_stopwords(text):
    text = [i for i in text if i not in stopwords]
    return text
reviews_sample['tokenized_text']  = reviews_sample['tokenized_text'].apply(remove_stopwords)

In [28]:
#stemming
from nltk.stem.porter import PorterStemmer
porterstemmer = PorterStemmer()
def stemming(text):
    text =[porterstemmer.stem(word) for word in text]
    return text
reviews_sample['tokenized_text']  = reviews_sample['tokenized_text'].apply(stemming)

In [29]:
reviews_sample[['tokenized_text','clean_text']].iloc[0,:]

tokenized_text    [gift, sephora, complementari, sampl, goop, gl...
clean_text        i was gifted by sephora a complementary sample...
Name: 0, dtype: object